In [30]:
import urllib.request
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords
url = "https://medium.com/@srjoglekar246/first-time-with-kaggle-a-convnet-to-classify-toxic-comments-with-keras-ef84b6d18328"
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; WOW64; rv:23.0) Gecko/20100101 Firefox/23.0'}
req = urllib.request.Request(url=url, headers=headers)
html = urllib.request.urlopen(req).read()
soup = BeautifulSoup(html, "lxml")
stop = stopwords.words('english')

In [25]:
[s.extract() for s in soup('script')]
[s.extract() for s in soup('style')]
str_body = soup.get_text()
str_body = re.sub(r'\d|\$|\¥|\[|\]|\)|\(|\\', '', str_body)

In [33]:
import string
import nltk

movie_content4 = []
for i in str_body:
    i = re.sub(r'[0-9]|/d+', '',i)
    i = re.sub('\s+', '', str(i))
    if i != '' and len(i) <= 15 and re.search(r'[A-za-z]', i):
        movie_content4.append(i)
#词性标注
text = movie_content4
POS_tag = nltk.pos_tag(text)
#print(POS_tag)

from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
adjective_tags = ['JJ','JJR','JJS']
#根据词性进行词根还原
lemmatized_text = []
for word in POS_tag:
    if word[1] in adjective_tags:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0],pos="a")))
    else:
        lemmatized_text.append(str(wordnet_lemmatizer.lemmatize(word[0]))) #default POS = noun
#print (lemmatized_text)
#标注好的还原过的单词
POS_tag = nltk.pos_tag(lemmatized_text)
#print(POS_tag)

stopwords2 = []
wanted_POS = ['NN','NNS','NNP','NNPS','JJ','JJR','JJS','VBG','FW'] 
for word in POS_tag:
    if word[1] not in wanted_POS:
        stopwords2.append(word[0])

punctuations = list(str(string.punctuation))
stopwords2 = stopwords2 + punctuations

stopwords_plus = []
stopwords_plus = stopwords2 + stop
stopwords_plus = set(stopwords_plus)

processed_text = []
for word in lemmatized_text:
    if word not in stopwords_plus:
        processed_text.append(word)
    
vocabulary = list(set(processed_text))
import numpy as np
import math
vocab_len = len(vocabulary)

weighted_edge = np.zeros((vocab_len,vocab_len),dtype=np.float32)
score = np.zeros((vocab_len),dtype=np.float32)
window_size = 3
covered_coocurrences = []

for i in range(0,vocab_len):
    score[i]=1
    for j in range(0,vocab_len):
        if j==i:
            weighted_edge[i][j]=0
        else:
            for window_start in range(0,(len(processed_text)-window_size)):
                window_end = window_start+window_size
                window = processed_text[window_start:window_end]
                if (vocabulary[i] in window) and (vocabulary[j] in window):
                    
                    index_of_i = window_start + window.index(vocabulary[i])
                    index_of_j = window_start + window.index(vocabulary[j])
                    
                    # index_of_x is the absolute position of the xth term in the window 
                    # (counting from 0) 
                    # in the processed_text
                      
                    if [index_of_i,index_of_j] not in covered_coocurrences:
                        weighted_edge[i][j]+=1/math.fabs(index_of_i-index_of_j)
                        covered_coocurrences.append([index_of_i,index_of_j])

inout = np.zeros((vocab_len),dtype=np.float32)

for i in range(0,vocab_len):
    for j in range(0,vocab_len):
        inout[i]+=weighted_edge[i][j]
        
MAX_ITERATIONS = 50
d=0.85
threshold = 0.0001 #convergence threshold

for iter in range(0,MAX_ITERATIONS):
    prev_score = np.copy(score)
    
    for i in range(0,vocab_len):
        
        summation = 0
        for j in range(0,vocab_len):
            if weighted_edge[i][j] != 0:
                summation += (weighted_edge[i][j]/inout[j])*score[j]
                
        score[i] = (1-d) + d*(summation)
    
    if np.sum(np.fabs(prev_score-score)) <= threshold: #convergence condition
        print ("Converging at iteration "+str(iter)+"....")
        break
        
phrases = []

phrase = " "
for word in lemmatized_text:
    
    if word in stopwords2:
        if phrase!= " ":
            phrases.append(str(phrase).strip().split())
        phrase = " "
    elif word not in stopwords2:
        phrase+=str(word)
        phrase+=" "

#print ("Partitioned Phrases (Candidate Keyphrases): \n")
print (phrases)

unique_phrases = []

for phrase in phrases:
    if phrase not in unique_phrases:
        unique_phrases.append(phrase)
        
for word in vocabulary:
    #print word
    for phrase in unique_phrases:
        if (word in phrase) and ([word] in unique_phrases) and (len(phrase)>1):
            unique_phrases.remove([word])

phrase_scores = []
keywords = []
for phrase in unique_phrases:
    phrase_score=0
    keyword = ''
    for word in phrase:
        keyword += str(word)
        keyword += " "
        phrase_score+=score[vocabulary.index(word)]
    phrase_scores.append(phrase_score)
    keywords.append(keyword.strip())

sorted_index = np.flip(np.argsort(phrase_scores),0)

keywords_num = 30

print ("Keywords:\n")

for i in range(0,keywords_num):
    print (str(i) + str(keywords[sorted_index[i]])+", ",)

Converging at iteration 25....
[['F'], ['K'], ['C'], ['N'], ['K'], ['H'], ['B'], ['S'], ['G'], ['H'], ['M'], ['S'], ['J'], ['B'], ['U'], ['F'], ['F'], ['E'], ['G'], ['M'], ['L'], ['J'], ['J'], ['F'], ['K'], ['C'], ['N'], ['K'], ['W'], ['K'], ['J'], ['T'], ['C'], ['C'], ['C'], ['M'], ['T'], ['C'], ['N'], ['C'], ['N'], ['K'], ['M', 'L', 'P'], ['T'], ['T'], ['C', 'S', 'V'], ['E'], ['H'], ['C', 'S', 'V'], ['C', 'S', 'V'], ['H'], ['T'], ['B'], ['N', 'L', 'T', 'K', 'T'], ['K'], ['T'], ['S'], ['j'], ['T'], ['M'], ['M'], ['X'], ['S', 'E', 'Q', 'U', 'E', 'N', 'C', 'E'], ['L', 'E', 'N', 'G', 'T', 'H', 'V'], ['L'], ['T'], ['O', 'N'], ['S', 'P', 'L'], ['T'], ['N'], ['N'], ['T'], ['M'], ['X'], ['S', 'E', 'Q', 'U', 'E', 'N', 'C', 'E'], ['L', 'E', 'N', 'G', 'T', 'H'], ['V'], ['L'], ['T'], ['O', 'N'], ['S', 'P', 'L'], ['T'], ['W'], ['E'], ['F'], ['G'], ['T'], ['O'], ['W'], ['V'], ['F'], ['W'], ['V'], ['G'], ['F'], ['F'], ['O', 'O', 'V'], ['G'], ['P'], ['G'], ['O'], ['K'], ['V'], ['K'], ['V'], ['j'], [

IndexError: index 24 is out of bounds for axis 0 with size 24